In [2]:
##############################################################################
# create SparkSession
##############################################################################

import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

PYSPARK_SUBMIT_ARGS = """--num-executors 3 \
--packages org.elasticsearch:elasticsearch-spark-20_2.11:7.7.0,\
com.datastax.spark:spark-cassandra-connector_2.11:2.4.3,\
org.postgresql:postgresql:42.2.12 pyspark-shell"""

os.environ["PYSPARK_SUBMIT_ARGS"] = PYSPARK_SUBMIT_ARGS

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ivashnikov").getOrCreate()
sc = spark.sparkContext

In [4]:
##############################################################################
# импорт необходимых библиотек и создание вспомогательных функций
##############################################################################

from pyspark.sql.functions import explode, col, expr, lower, concat, lit, create_map, regexp_replace
from itertools import chain
from auth_postgresql import login, password


def open_elastic(table_name, option=("es.nodes", "10.0.0.5:9200")):
    df = spark.read.format("org.elasticsearch.spark.sql").option(*option).load(table_name)
    return df


def open_cassandra(table_opts, host='10.0.0.5', port='9042'):
    spark.conf.set("spark.cassandra.connection.host", host)
    spark.conf.set("spark.cassandra.connection.port", port)
    spark.conf.set("spark.cassandra.output.consistency.level", "ANY")
    spark.conf.set("spark.cassandra.input.consistency.level", "ONE")

    df = spark.read.format("org.apache.spark.sql.cassandra").options(**table_opts).load()
    return df


def open_json(path):
    df = spark.read.format('json').load(path)
    return df


def open_postgresql(table_name, jdbc_url):
    df = spark.read.format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", table_name) \
        .option("driver", "org.postgresql.Driver") \
        .load()
    return df


##############################################################################
# загрузка данных
##############################################################################

# Информация о клиентах
table_opts = {"table": "clients", "keyspace": "labdata"}
clients = open_cassandra(table_opts)

# Логи посещения интернет-магазина
visits = open_elastic("visits")

# Логи посещения веб-сайтов
weblogs = open_json('/mf-labs/laba02/weblogs.json')

# Информация о категориях веб-сайтов
jdbc_url = f'jdbc:postgresql://10.0.0.5:5432/labdata?user={login}&password={password}'
domain_cats = open_postgresql('domain_cats', jdbc_url)


##############################################################################
# обработка данных
##############################################################################

# обработка данных о клиентах
age_cat_dict = {i:
    '18-24' if 18 <= i < 25 else
    '25-34' if 25 <= i < 35 else 
    '35-44' if 35 <= i < 45 else 
    '45-54' if 45 <= i < 55 else 
    '>=55'
    for i in range(18, 100)
}
mapping_expr = create_map([lit(x) for x in chain(*age_cat_dict.items())])
clients_processed = clients.select('uid', 'gender', mapping_expr[col('age')].alias('age_cat'))

# обработка данных о посещениях интернет магазина
cat = concat(lit('shop_'), lower(regexp_replace(col('category'), '\s|-', '_')))
visits_count = visits \
    .na.drop(subset='uid') \
    .withColumn('cat', cat) \
    .groupby('uid') \
    .pivot('cat') \
    .count() \
    .na.fill(0)
visits_count.cache()

# обработка данных о посещениях веб-сайтов
domain_cats_processed = domain_cats.withColumn('category', concat(lit('web_'), 'category'))
domain = expr("regexp_replace(parse_url(visits.url, 'HOST'), '^www\.', '')").alias('domain')
weblogs_count = weblogs \
    .withColumn('visits', explode('visits')) \
    .select('uid', domain) \
    .join(domain_cats_processed, on='domain', how='inner') \
    .groupby('uid') \
    .pivot('category') \
    .count() \
    .na.fill(0)
weblogs_count.cache()

DataFrame[uid: string, web_arts_and_entertainment: bigint, web_autos_and_vehicles: bigint, web_beauty_and_fitness: bigint, web_books_and_literature: bigint, web_business_and_industry: bigint, web_career_and_education: bigint, web_computer_and_electronics: bigint, web_finance: bigint, web_food_and_drink: bigint, web_gambling: bigint, web_games: bigint, web_health: bigint, web_home_and_garden: bigint, web_internet_and_telecom: bigint, web_law_and_government: bigint, web_news_and_media: bigint, web_pets_and_animals: bigint, web_recreation_and_hobbies: bigint, web_reference: bigint, web_science: bigint, web_shopping: bigint, web_sports: bigint, web_travel: bigint]

In [9]:
weblogs_count.show(1, vertical=True)

-RECORD 0--------------------------------------------
 uid                          | 6f4d388e-330c-44b... 
 web_arts_and_entertainment   | 1                    
 web_autos_and_vehicles       | 0                    
 web_beauty_and_fitness       | 0                    
 web_books_and_literature     | 0                    
 web_business_and_industry    | 0                    
 web_career_and_education     | 0                    
 web_computer_and_electronics | 0                    
 web_finance                  | 0                    
 web_food_and_drink           | 0                    
 web_gambling                 | 0                    
 web_games                    | 0                    
 web_health                   | 0                    
 web_home_and_garden          | 0                    
 web_internet_and_telecom     | 0                    
 web_law_and_government       | 0                    
 web_news_and_media           | 4                    
 web_pets_and_animals       

In [16]:
# создание финальной таблицы и запись в БД
clients_stat = clients_processed \
    .join(visits_count, on='uid', how='left') \
    .join(weblogs_count, on='uid', how='left')
clients_stat.cache()

DataFrame[uid: string, gender: string, age_cat: string, shop_cameras: bigint, shop_clothing: bigint, shop_computers: bigint, shop_cosmetics: bigint, shop_entertainment_equipment: bigint, shop_everyday_jewelry: bigint, shop_house_repairs_paint_tools: bigint, shop_household_appliances: bigint, shop_household_furniture: bigint, shop_kitchen_appliances: bigint, shop_kitchen_utensils: bigint, shop_luggage: bigint, shop_mobile_phones: bigint, shop_shoes: bigint, shop_sports_equipment: bigint, shop_toys: bigint, web_arts_and_entertainment: bigint, web_autos_and_vehicles: bigint, web_beauty_and_fitness: bigint, web_books_and_literature: bigint, web_business_and_industry: bigint, web_career_and_education: bigint, web_computer_and_electronics: bigint, web_finance: bigint, web_food_and_drink: bigint, web_gambling: bigint, web_games: bigint, web_health: bigint, web_home_and_garden: bigint, web_internet_and_telecom: bigint, web_law_and_government: bigint, web_news_and_media: bigint, web_pets_and_an

In [18]:
clients_stat.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://10.0.0.5:5432/dmitry_ivashnikov") \
    .option("dbtable", "clients") \
    .option("user", login) \
    .option("password", password) \
    .option("driver", "org.postgresql.Driver") \
    .save()

In [4]:
clients_stat.count()

22884

In [17]:
clients_processed.groupby('gender', 'age_cat').count().show()

+------+-------+-----+
|gender|age_cat|count|
+------+-------+-----+
|     F|  25-34| 6791|
|     F|  45-54| 2597|
|     M|  45-54| 2147|
|     M|  35-44| 5089|
|     F|   >=55|  895|
|     M|   >=55|  784|
|     F|  18-24| 2886|
|     M|  25-34| 8666|
|     M|  18-24| 2012|
|     F|  35-44| 4271|
+------+-------+-----+



In [13]:
clients_stat.groupby('gender', 'age_cat').count().show()

+------+-------+-----+
|gender|age_cat|count|
+------+-------+-----+
|     F|  25-34| 6791|
|     F|  45-54| 2362|
|     M|  45-54| 1940|
|     M|  35-44| 5089|
|     F|   >=55| 1130|
|     M|   >=55|  991|
|     M|  25-34| 8666|
|     F|  18-24| 2886|
|     M|  18-24| 2012|
|     F|  35-44| 4271|
+------+-------+-----+



In [2]:
clients_stat.show(1, vertical=True)

-RECORD 0----------------------------------------------
 uid                            | 0392f398-ea7e-4a1... 
 gender                         | F                    
 age_cat                        | >=55                 
 shop_cameras                   | 1                    
 shop_clothing                  | 0                    
 shop_computers                 | 1                    
 shop_cosmetics                 | 0                    
 shop_entertainment_equipment   | 1                    
 shop_everyday_jewelry          | 0                    
 shop_house_repairs_paint_tools | 0                    
 shop_household_appliances      | 0                    
 shop_household_furniture       | 0                    
 shop_kitchen_appliances        | 0                    
 shop_kitchen_utensils          | 0                    
 shop_luggage                   | 0                    
 shop_mobile_phones             | 1                    
 shop_shoes                     | 0             

In [ ]:
# GRANT SELECT ON TABLE clients TO labchecker2;